In [20]:
pip -q install tweepy==4.10.0 openai==1.27.0 diffusers k-diffusion peft torchvision==0.16.0 git+https://github.com/sberbank-ai/Real-ESRGAN.git runpod

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 2.6 MB/s eta 0:00:00


In [3]:
### OpenAI for translation
import openai

api_key = 'OPEN_AI_KEY'
openai_client = openai.Client(api_key=api_key)

def translate_text(text):
    response = openai_client.chat.completions.create(
              messages=[
              {
              "role": "user",
              "content": "Translate the given text to english. Text: {}".format(text)
              }
              ],model="gpt-3.5-turbo")
    return response.choices[0].message.content

# Example usage
text_to_translate = "Che bellissima giornat"
translated_text = translate_text(text_to_translate)
print(f'Translated Text: {text_to_translate, translated_text}')

Translated Text: ('Che bellissima giornat', 'What a beautiful day')


In [4]:
### Stable Diffusion
from transformers import pipeline
import torch
import peft
from diffusers import StableDiffusionKDiffusionPipeline, StableDiffusionImg2ImgPipeline
from RealESRGAN import RealESRGAN
import os

access_token = "HUGGINGFACE_KEY"

def image_gen(prompts,device):
    ### Load model file
    pipe = StableDiffusionKDiffusionPipeline.from_pretrained('ai-chatbot-degen/yayoiMix_v25',
                                                             torch_dtype=torch.float16,
                                                             safety_checker=None,
                                                            use_auth_token=access_token).to(device)

    ### Scheduler or Sampler
    pipe.set_scheduler('sample_dpmpp_2m_sde')

    ### Negative prompt embedding
    pipe.load_textual_inversion("ai-chatbot-degen/yayoiMix_v25",
                                        weight_name="easynegative.safetensors",
                                        token="easynegative",
                                        force_download=True)
    pipe.load_textual_inversion("ai-chatbot-degen/yayoiMix_v25",
                                        weight_name="badhandv4.pt",
                                        token="badhand",
                                        force_download=True)
    pipe.load_textual_inversion("ai-chatbot-degen/yayoiMix_v25",
                                        weight_name="negative_hand-neg.pt",
                                        token="negativehand",
                                        force_download=True)
    pipe.load_textual_inversion("ai-chatbot-degen/yayoiMix_v25",
                                        weight_name="bad-hands-5.pt",
                                        token="badhand5",
                                        force_download=True)

    # Hyperparamters
    num_images_per_prompt = 1
    height = 880
    width = 544

    # Seed parameter
    generator=torch.Generator(device=device)
    seeds = []

    for _ in range(num_images_per_prompt):
        seed = generator.seed()
        seeds.append(seed)
        generator = generator.manual_seed(seed)

    prompts = prompts
    negative_prompts = '''
        Negative prompt: painting,sketches,(worst quality:2),(low quality:2),(normal quality:2),((monochrome)),
        ((grayscale)), missing fingers ,skin spots ,acnes,skin blemishes,loli, easynegative, Bad-Hands-5, negative_hand
        '''
    num_inference_steps = 33
    guidance_scale = 7
    clip_skip = 1
    ### Image generation
    out = pipe(prompts,
        generator=generator,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        num_images_per_prompt=1,
        negative_prompt=negative_prompts,
        height=height,
        width=width,
        clip_skip=clip_skip,
        use_karras_sigmas=True).images

    ### Denoising
    img2img = StableDiffusionImg2ImgPipeline(**pipe.components)
    res_image = img2img(prompt=prompts,
        negative_prompt=negative_prompts,
        generator=generator,
        num_inference_steps=num_inference_steps*3,
        image = out[0].resize((width*1, height*1)),
        strength = 0.45,
        guidance_scale=guidance_scale,
        clip_skip=clip_skip
        ).images

    ### Upscaler
    model1 = RealESRGAN(device="cuda", scale=8)
    model1.load_weights('weights/RealESRGAN_x8.pth', download=True)
    sr_image = model1.predict(res_image[0])

    return sr_image.resize((320,520))

/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


In [5]:
from huggingface_hub import login
login(token='HUGGINGFACE_KEY')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
### Runpod
import runpod

runpod.api_key = "RUNPOD_KEY"
endpoints = runpod.get_endpoints()
# print(endpoints)

endpoint = runpod.Endpoint("ENDPOINTS")
output = image_gen('Hi world','cuda')
output

In [1]:
API_KEY = 'API'
API_KEY_SECRET = 'SECRET'
ACCESS_TOKEN = 'TOKEN'
ACCESS_TOKEN_SECRET = 'TOEKN_SECRET'
BEARER_TOKEN = 'BEARER'

In [2]:
import tweepy
import os

consumer_key = API_KEY
consumer_secret = API_KEY_SECRET
access_token = ACCESS_TOKEN
access_token_secret = ACCESS_TOKEN_SECRET
bearer_token = BEARER_TOKEN

auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth=auth, wait_on_rate_limit=True)
client = tweepy.Client(bearer_token, consumer_key, consumer_secret, access_token, access_token_secret, wait_on_rate_limit=True)

In [3]:
### Get own user account
client_id = client.get_me().data.id

In [ ]:
### Fetch latest tweets
start_id = 1
initialisation_resp = client.get_users_mentions(client_id)
if initialisation_resp.data != None:
    start_id = initialisation_resp.data[0].id

In [ ]:
### Automated replies
while True:
    response = client.get_users_mentions(client_id, since_id=start_id)

    # Reply Code
    if response.data != None:
        for tweet in response.data:
            try:
                message = image_gen(translate_text(tweet.text),'cuda')
                # client.create_tweet(in_reply_to_tweet_id=tweet.id, text='Follow me for more')
                api.update_with_media(filename = message, status = 'Follow me for more', in_reply_to_status_id = tweet.id)
                start_id = tweet.id
            except Exception as error:
                print(error)

    time.sleep(15)